In [40]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
from pandas import Series
import numpy as np

import matplotlib.pylab as plt
%matplotlib inline

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6
import re
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
import statsmodels.api as sm
import itertools

In [123]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
*The raw code for this Jupyter notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [50]:
import pandas as pd
import matplotlib.pyplot as plt
from pandas import Series
import numpy as np
import itertools
import matplotlib.pylab as plt
%matplotlib inline
import re
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
import statsmodels.api as sm
import itertools
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6

In [63]:
index_for_df = [pd.to_datetime('2018-09-04 10:15:00')+ datetime.timedelta(minutes=5*x) for x in range(0, 72)]

In [64]:
def get_all_predictions(dataframe):
    pred_df = pd.DataFrame(index = index_for_df)
    p = d = q = range(0, 2)
    pdq = list(itertools.product(p, d, q))
    pdqs = [(x[0], x[1], x[2], 1) for x in list(itertools.product(p, d, q))]
    df = dataframe.drop('5. volume', axis = 1)
    for metric in df.columns:
        ans = []
        for comb in pdq:
            for combs in pdqs:
                try:
                    mod = sm.tsa.statespace.SARIMAX(dataframe[metric],
                                            order=comb,
                                            seasonal_order=combs,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)
                    output = mod.fit()
                    ans.append([comb, combs, output.aic])
                except:
                    continue
        ans_df = pd.DataFrame(ans, columns=['pdq', 'pdqs', 'aic'])
        ARIMA_MODEL = sm.tsa.statespace.SARIMAX(dataframe[metric],
                                print(ans_df),
                                order=ans_df.loc[ans_df['aic'].idxmin()][0],
                                seasonal_order=ans_df.loc[ans_df['aic'].idxmin()][1],
                                enforce_stationarity=False,
                                enforce_invertibility=False)
        output = ARIMA_MODEL.fit()
        pred = output.get_prediction(start=pd.to_datetime('2018-09-04 10:15:00'), end=pd.to_datetime('2018-09-04 15:55:00'),dynamic=False)
        predictions = pd.DataFrame(pred.predicted_mean)
        pred_df[metric] = predictions
    return pred_df

In [18]:
# index_for_df = apple.index[(apple.index < '2018-08-31 15:55:00') & (apple.index > '2018-08-31 09:55:00')]

In [20]:
import datetime

In [42]:
import alpha_vantage
from alpha_vantage.timeseries import TimeSeries
stock = TimeSeries(key='1250F9WWA3Z77BIK')

In [43]:
fb_sept, meta_sept = stock.get_intraday(symbol ='FB', interval='5min')

In [44]:
fb_today = pd.DataFrame(fb_sept)
fb_today = pd.DataFrame(fb_today.T)
fb_today.index = pd.to_datetime(fb_today.index)

In [45]:
fb_today_graph = fb_today[(fb_today.index > '2018-09-04')]

In [11]:
nflx = pd.read_csv('./csv/nflx_daily_stock.csv', index_col='Unnamed: 0')
fb = pd.read_csv('./csv/fb_daily_stock.csv', index_col='Unnamed: 0')
apple = pd.read_csv('./csv/apple_daily_stock.csv', index_col='Unnamed: 0')
fb_prediction_df = pd.read_csv('./csv/fb_predictions.csv', index_col='Unnamed: 0')

In [65]:
fb_today_pred = get_all_predictions(fb_today)

Empty DataFrame
Columns: [pdq, pdqs, aic]
Index: []


TypeError: reduction operation 'argmin' not allowed for this dtype

# Candlestick Charts 8-31-2018

In [6]:
import plotly.plotly as py
import plotly.graph_objs as go

from datetime import datetime

# ts_close = pd.DataFrame(fb['4. close'])
# ts_close = ts_close['4. close'].astype(float)
# rolmean = ts_close.rolling(window=6).mean()
# orig_close = plt.plot(ts, color='blue',label='Original')
# mean_close = plt.plot(rolmean, color='red', label='Rolling Mean')
#     std = plt.plot(rolstd, color='black', label = 'Rolling Std')
# plt.legend(loc='best')
# plt.title('Rolling Mean & Standard Deviation')
# plt.show(block=False)


layout = {'xaxis': {'rangeslider':{'visible':False}},
    'title': 'Facebook Stock 08-31-2018',
    'yaxis': {'title': 'Stock Price'},'shapes': [{
        'x0': '2018-08-31 14:00:00', 'x1': '2018-08-31 14:00:00',
        'y0': 0, 'y1': 1, 'xref': 'x', 'yref': 'paper',
        'line': {'color': 'rgb(30,30,30)', 'width': 1}
    }],'annotations': [{
        'x': '2018-08-31 14:00:00', 'y': 0.05, 'xref': 'x', 'yref': 'paper',
        'showarrow': False, 'xanchor': 'left',
        'text': 'Predictions Begin'
    }]}


trace = go.Candlestick(name = 'Price',x=nflx.index,
                       open=fb_today_graph['1. open'],
                       high=fb_today_graph['2. high'],
                       low=fb_today_graph['3. low'],
                       close=fb_today_graph['4. close'],
                       increasing=dict(line=dict(color= 'Blue')),
                       decreasing=dict(line=dict(color= 'Silver')))

trace2 = go.Candlestick(name = 'Predicted Price',x=fb_prediction_df.index,
                       open=fb_prediction_df.open_pred,
                       high=fb_prediction_df.high_pred,
                       low=fb_prediction_df.low_pred,
                       close=fb_prediction_df.close_pred,
                       increasing=dict(line=dict(color= 'Green')),
                       decreasing=dict(line=dict(color= 'Red')))

# trace2 = go.Scatter(name = 'Close Price MA',
#     x = rolmean.index,
#     y = rolmean.values, line = dict(
#         color = ('red'), o))

data = [trace, trace2]
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='plot from API (15)')

In [93]:
apple_prediction_df = get_all_predictions(apple)

In [98]:
apple_prediction_df.to_csv('apple_predictions.csv')

In [97]:
import plotly.plotly as py
import plotly.graph_objs as go

from datetime import datetime


layout = {'xaxis': {'rangeslider':{'visible':False}},
    'title': 'Apple Stock 08-31-2018',
    'yaxis': {'title': 'Stock Price'},'shapes': [{
        'x0': '2018-08-31 14:00:00', 'x1': '2018-08-31 14:00:00',
        'y0': 0, 'y1': 1, 'xref': 'x', 'yref': 'paper',
        'line': {'color': 'rgb(30,30,30)', 'width': 1}
    }],'annotations': [{
        'x': '2018-08-31 14:00:00', 'y': 0.05, 'xref': 'x', 'yref': 'paper',
        'showarrow': False, 'xanchor': 'left',
        'text': 'Predictions Begin'
    }]}


trace = go.Candlestick(name = 'Actual Price',x=apple.index,
                       open=apple['1. open'],
                       high=apple['2. high'],
                       low=apple['3. low'],
                       close=apple['4. close'],
                       increasing=dict(line=dict(color= 'Black')),
                       decreasing=dict(line=dict(color= 'Silver')))

trace2 = go.Candlestick(name = 'Predicted Price',x=apple_prediction_df.index,
                       open=apple_prediction_df['1. open'],
                       high=apple_prediction_df['2. high'],
                       low=apple_prediction_df['3. low'],
                       close=apple_prediction_df['4. close'],
                       increasing=dict(line=dict(color= 'Green')),
                       decreasing=dict(line=dict(color= 'Red')))


# trace2 = go.Scatter(name = 'Close Price MA',
#     x = rolmean.index,
#     y = rolmean.values, line = dict(
#         color = ('red'), o))

data = [trace,trace2]
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='plot from API (16)')

In [118]:
netflix_prediction_df = get_all_predictions(nflx)

In [121]:
netflix_prediction_df.to_csv('netflix_predictions.csv')

In [120]:
import plotly.plotly as py
import plotly.graph_objs as go

from datetime import datetime

layout = {'xaxis': {'rangeslider':{'visible':False}},
    'title': 'Netflix Stock 08-31-2018',
    'yaxis': {'title': 'Stock Price'},'shapes': [{
        'x0': '2018-08-31 14:00:00', 'x1': '2018-08-31 14:00:00',
        'y0': 0, 'y1': 1, 'xref': 'x', 'yref': 'paper',
        'line': {'color': 'rgb(30,30,30)', 'width': 1}
    }],'annotations': [{
        'x': '2018-08-31 14:00:00', 'y': 0.05, 'xref': 'x', 'yref': 'paper',
        'showarrow': False, 'xanchor': 'left',
        'text': 'Predictions Begin'
    }]}

trace = go.Candlestick(name = 'Actual Price',x=nflx.index,
                       open=nflx['1. open'],
                       high=nflx['2. high'],
                       low=nflx['3. low'],
                       close=nflx['4. close'],
                       increasing=dict(line=dict(color= 'Red')),
                       decreasing=dict(line=dict(color= 'Black')))

trace2 = go.Candlestick(name = 'Predicted Price',x=netflix_prediction_df.index,
                       open=netflix_prediction_df['1. open'],
                       high=netflix_prediction_df['2. high'],
                       low=netflix_prediction_df['3. low'],
                       close=netflix_prediction_df['4. close'],
                       increasing=dict(line=dict(color= 'Green')),
                       decreasing=dict(line=dict(color= 'Silver')))

data = [trace, trace2]
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='plot from API (18)')